In [62]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

pd.set_option('max_columns',60)

In [63]:
AMZN_statement_folder = "../data/timeseries/AMZN/Statement"
file_list = []
for dirname, _, filenames in os.walk(f"{AMZN_statement_folder}"):
    for filename in filenames:
        file_list.append(os.path.join(dirname,filename))

list_num = 4
print(file_list[list_num])
df = pd.read_csv(file_list[list_num],index_col=[0,1])
df

IndexError: list index out of range

In [104]:
df = pd.read_csv('../data/timeseries/ZG/Statement/Condensed Consolidated Statements of Operations.csv',index_col=[0,1])

In [132]:
df[df.index.get_level_values(1).isin(['us-gaap_revenuefromcontractwithcustomerexcludingassessedtax'])]


,,2020-03-31,2019-03-31,2017-12-31,2018-12-31,2019-12-31,2018-09-30,2019-09-30,2018-06-30,2019-06-30,2018-03-31,2016-12-31,2017-09-30,2017-06-30
label,xbrl_tag,,,,,,,,,,,,,
"Revenue from Contract with Customer, Excluding Assessed Tax",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,1.125821e+09,454104000.0,1.076794e+09,1.333554e+09,2.742837e+09,343094000.0,745208000.0,325246000.0,599578000.0,NaN,846589000.0,281839000.0,266850000.0


In [84]:
csv_file ="../data/csv/UBER/2019-12-31 (10-K)/1005000 - Statement - CONSOLIDATED STATEMENTS OF CASH FLOWS.csv"


#csv_file ="../data/csv/UBER/2020-03-31 (10-Q)/1005000 - Statement - CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS.csv"
df = pd.read_csv(csv_file,index_col=[0])
len(df)
#df.index.isin(['us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal'])

75

In [86]:
len(np.unique(df.index))

75

In [89]:
df[~df.index.isin(['us-gaap_profitloss', 'us-gaap_depreciationdepletionandamortization', 'us-gaap_provisionfordoubtfulaccounts', 'us-gaap_sharebasedcompensation', 'us-gaap_gainslossesonextinguishmentofdebt', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_deferredincometaxexpensebenefit', 'us-gaap_embeddedderivativegainlossonembeddedderivativenet', 'us-gaap_amortizationofdebtdiscountpremium', 'us-gaap_paidinkindinterest', 'us-gaap_gainlossonsaleofpropertyplantequipment', 'us-gaap_impairmentoflonglivedassetstobedisposedof', 'us-gaap_incomelossfromequitymethodinvestments', 'us-gaap_debtandequitysecuritiesunrealizedgainloss', 'us-gaap_increasedecreaseincontractwithcustomerliability', 'us-gaap_fairvalueadjustmentofwarrants', 'us-gaap_foreigncurrencytransactiongainlossunrealized', 'us-gaap_othernoncashincomeexpense', 'us-gaap_increasedecreaseinaccountsreceivable', 'us-gaap_increasedecreaseinprepaiddeferredexpenseandotherassets', 'uber_increasedecreasefundsheldbyinsurers', 'uber_increasedecreaseinoperatingleaserightofuseassets', 'us-gaap_increasedecreaseinaccountspayable', 'uber_increasedecreaseininsurancereserve', 'us-gaap_increasedecreaseinaccruedliabilitiesandotheroperatingliabilities', 'uber_increasedecreaseinoperatingleaseliabilities', 'us-gaap_netcashprovidedbyusedinoperatingactivities', 'us-gaap_proceedsfromsaleofpropertyplantandequipment', 'us-gaap_paymentstoacquirepropertyplantandequipment', 'us-gaap_paymentstoacquireintangibleassets', 'us-gaap_paymentstoacquireequitymethodinvestments', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'uber_paymentstoacquirenonmarketableinvestments', 'us-gaap_paymentstoacquiremarketablesecurities', 'us-gaap_proceedsfromsaleandmaturityofmarketablesecurities', 'us-gaap_proceedsfromdivestitureofbusinessesnetofcashdivested', 'us-gaap_paymentstoacquirebusinessesnetofcashacquired', 'us-gaap_netcashprovidedbyusedininvestingactivities', 'us-gaap_proceedsfromissuanceinitialpublicoffering', 'us-gaap_paymentsrelatedtotaxwithholdingforsharebasedcompensation', 'us-gaap_proceedsfromissuanceofprivateplacement', 'us-gaap_proceedsfromissuanceofpreferredstockandpreferencestock', 'us-gaap_proceedsfromstockoptionsexercised', 'uber_proceedsfromissuanceofcommonstockemployeestockpurchaseplan', 'us-gaap_paymentsforrepurchaseofequity', 'us-gaap_proceedsfromissuanceoflongtermdebt', 'us-gaap_repaymentsoflongtermdebt', 'us-gaap_proceedsfromlinesofcredit', 'us-gaap_repaymentsoflinesofcredit', 'us-gaap_repaymentsoflongtermcapitalleaseobligations', 'us-gaap_financeleaseprincipalpayments', 'us-gaap_proceedsfromissuanceofredeemableconvertiblepreferredstock', 'uber_proceedsfromdissolutionofjointventureandsubsequentproceeds', 'uber_repurchaseofstocksubjecttoputoption', 'us-gaap_proceedsfrompaymentsforotherfinancingactivities', 'us-gaap_netcashprovidedbyusedinfinancingactivities', 'us-gaap_effectofexchangerateoncashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsperiodincreasedecreaseincludingexchangerateeffect', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsincludingdisposalgroupanddiscontinuedoperations', 'uber_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsreclassificationfromtoassetsheldforsaleduringperiod', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_interestpaidnet', 'us-gaap_incometaxespaidnet', 'us-gaap_conversionofstockamountconverted1', 'us-gaap_debtconversionconvertedinstrumentamount1', 'uber_sharebasedcompensationcapitalizedassoftwaredevelopmentcosts', 'uber_increasedecreaseincapitalassetsincurredbutnotyetpaid', 'uber_increasedecreaseinsharerepurchasecommitmentincurredbutnotyetpaid', 'us-gaap_constructioninprogressexpendituresincurredbutnotyetpaid', 'us-gaap_capitalleaseobligationsincurred', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'us-gaap_stockissued1', 'uber_ownershipinterestacquiredthroughdissolutionofjointventure', 'us-gaap_noncashorpartnoncashdivestitureamountofconsiderationreceived1'])]

,terseLabel,period_type,label,2019-12-31,2018-12-31,2017-12-31,2016-12-31
consolidatedstatementsofcashflows,,,,,,,
us-gaap_gainlossoninvestments,NaN,ytd,NaN,2000000.0,2.000000e+09,0.0,NaN
uber_proceedsfrominsurancereimbursementsaleanddisposalofpropertyandequipment,"Proceeds from insurance reimbursement, sale an...",ytd,"Proceeds From Insurance Reimbursement, Sale An...",51000000.0,3.690000e+08,342000000.0,NaN
uber_paymentstoacquiredebtsecuritiesnotreadilymarketable,NaN,ytd,"Payments To Acquire Debt Securities, Not Readi...",0.0,3.000000e+07,0.0,NaN
uber_deferredunpaidofferingcosts,Deferred unpaid offering costs,ytd,Deferred Unpaid Offering Costs,0.0,4.000000e+06,0.0,NaN
uber_litigationsettlementthroughstockissuance,Settlement of litigation through issuance of r...,ytd,Litigation Settlement Through Stock Issuance,0.0,2.500000e+08,0.0,NaN


In [96]:
df.index.isin(['us-gaap_profitloss', 'us-gaap_depreciationdepletionandamortization', 'us-gaap_provisionfordoubtfulaccounts', 'us-gaap_sharebasedcompensation', 'us-gaap_gainslossesonextinguishmentofdebt', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_deferredincometaxexpensebenefit', 'us-gaap_embeddedderivativegainlossonembeddedderivativenet', 'us-gaap_amortizationofdebtdiscountpremium', 'us-gaap_paidinkindinterest', 'us-gaap_gainlossonsaleofpropertyplantequipment', 'us-gaap_impairmentoflonglivedassetstobedisposedof', 'us-gaap_incomelossfromequitymethodinvestments', 'us-gaap_debtandequitysecuritiesunrealizedgainloss', 'us-gaap_increasedecreaseincontractwithcustomerliability', 'us-gaap_fairvalueadjustmentofwarrants', 'us-gaap_foreigncurrencytransactiongainlossunrealized', 'us-gaap_othernoncashincomeexpense', 'us-gaap_increasedecreaseinaccountsreceivable', 'us-gaap_increasedecreaseinprepaiddeferredexpenseandotherassets', 'uber_increasedecreasefundsheldbyinsurers', 'uber_increasedecreaseinoperatingleaserightofuseassets', 'us-gaap_increasedecreaseinaccountspayable', 'uber_increasedecreaseininsurancereserve', 'us-gaap_increasedecreaseinaccruedliabilitiesandotheroperatingliabilities', 'uber_increasedecreaseinoperatingleaseliabilities', 'us-gaap_netcashprovidedbyusedinoperatingactivities', 'us-gaap_proceedsfromsaleofpropertyplantandequipment', 'us-gaap_paymentstoacquirepropertyplantandequipment', 'us-gaap_paymentstoacquireintangibleassets', 'us-gaap_paymentstoacquireequitymethodinvestments', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'uber_paymentstoacquirenonmarketableinvestments', 'us-gaap_paymentstoacquiremarketablesecurities', 'us-gaap_proceedsfromsaleandmaturityofmarketablesecurities', 'us-gaap_proceedsfromdivestitureofbusinessesnetofcashdivested', 'us-gaap_paymentstoacquirebusinessesnetofcashacquired', 'us-gaap_netcashprovidedbyusedininvestingactivities', 'us-gaap_proceedsfromissuanceinitialpublicoffering', 'us-gaap_paymentsrelatedtotaxwithholdingforsharebasedcompensation', 'us-gaap_proceedsfromissuanceofprivateplacement', 'us-gaap_proceedsfromissuanceofpreferredstockandpreferencestock', 'us-gaap_proceedsfromstockoptionsexercised', 'uber_proceedsfromissuanceofcommonstockemployeestockpurchaseplan', 'us-gaap_paymentsforrepurchaseofequity', 'us-gaap_proceedsfromissuanceoflongtermdebt', 'us-gaap_repaymentsoflongtermdebt', 'us-gaap_proceedsfromlinesofcredit', 'us-gaap_repaymentsoflinesofcredit', 'us-gaap_repaymentsoflongtermcapitalleaseobligations', 'us-gaap_financeleaseprincipalpayments', 'us-gaap_proceedsfromissuanceofredeemableconvertiblepreferredstock', 'uber_proceedsfromdissolutionofjointventureandsubsequentproceeds', 'uber_repurchaseofstocksubjecttoputoption', 'us-gaap_proceedsfrompaymentsforotherfinancingactivities', 'us-gaap_netcashprovidedbyusedinfinancingactivities', 'us-gaap_effectofexchangerateoncashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsperiodincreasedecreaseincludingexchangerateeffect', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsincludingdisposalgroupanddiscontinuedoperations', 'uber_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsreclassificationfromtoassetsheldforsaleduringperiod', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_interestpaidnet', 'us-gaap_incometaxespaidnet', 'us-gaap_conversionofstockamountconverted1', 'us-gaap_debtconversionconvertedinstrumentamount1', 'uber_sharebasedcompensationcapitalizedassoftwaredevelopmentcosts', 'uber_increasedecreaseincapitalassetsincurredbutnotyetpaid', 'uber_increasedecreaseinsharerepurchasecommitmentincurredbutnotyetpaid', 'us-gaap_constructioninprogressexpendituresincurredbutnotyetpaid', 'us-gaap_capitalleaseobligationsincurred', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'us-gaap_stockissued1', 'uber_ownershipinterestacquiredthroughdissolutionofjointventure', 'us-gaap_noncashorpartnoncashdivestitureamountofconsiderationreceived1']
)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True])

In [97]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=6bd735b63675ca84457fe0eda0f77a2221fe3e8d54e94b19e3dabea8bc6e08da
  Stored in directory: /Users/goldbloom/Library/Caches/pip/wheels/28/79/3c/08a301b673366fe664fe2853e136eb2acc1bb752045c11c6a1
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8366 sha256=49661ddf7d0dac1188372e161d429c63b8ae541c84a65ed578ab64b26fd124a7
  Stored in directory: /Users/goldbloom/Library/Caches/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking
Note: you may need to restart the kernel to use updated packages.


In [103]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
print(msft.info['marketCap'])

1579251400704


In [100]:
msft.history(period="5d")


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-06-30,197.88,204.40,197.74,203.51,34310300,0,0
2020-07-01,203.14,206.35,201.77,204.70,32061200,0,0
2020-07-02,205.68,208.02,205.00,206.26,29315800,0,0
2020-07-06,208.83,211.13,208.09,210.70,31897600,0,0
2020-07-07,210.45,214.67,207.99,208.25,33530900,0,0
